In [1]:
import sys
sys.path.append('../src')

import os
import requests
import json
import pandas as pd
import betfairlightweight
from betfairlightweight import filters
from datetime import datetime, timedelta
import time

from betfair import Betfair

pd.set_option('display.max_columns', None)

In [2]:
app_key = 'ABGJLOlKaLtTsMIp'

username1 = input("Email:")
password1 = input("Password:")

In [3]:
trading = betfairlightweight.APIClient(
        username1, password1, app_key
    )
trading.login_interactive()

trading.session_token

trading.keep_alive()

<KeepAliveResource>

In [4]:
# Set the start and end time to cover the entire day in UTC
today = datetime.utcnow()
start_time = today.replace(hour=0, minute=0, second=0, microsecond=0)
end_time = today.replace(hour=23, minute=59, second=59, microsecond=999999)

In [5]:
# Define filter for horse racing markets
market_filter = betfairlightweight.filters.market_filter(
    event_type_ids=['7'],
    market_countries=['GB'],
    market_type_codes=['WIN'], 
    market_start_time={'from': start_time.isoformat(),
                       'to' : end_time.isoformat()}
)

# Request market catalogue for all horse races on the date
results = trading.betting.list_market_catalogue(
    filter=market_filter,
    market_projection=["RUNNER_DESCRIPTION", 
            "RUNNER_METADATA", 
            "COMPETITION", 
            "EVENT", 
            "EVENT_TYPE", 
            "MARKET_DESCRIPTION", 
            "MARKET_START_TIME",],  # Include runner metadata
    max_results=1000  # Adjust based on expected number of races
)

In [6]:
for i in results:
        print(
            "{0} {1:d}:{2:02d} {3} ({4})".format(
                i.market_id,
                i.market_start_time.hour, 
                i.market_start_time.minute, 
                i.event.venue,  
                i.description.market_type,
            )
        )


1.232219790 12:50 Chepstow (WIN)
1.232219921 13:02 Ripon (WIN)
1.232219882 13:07 Epsom (WIN)
1.232219964 13:12 Southwell (WIN)
1.232219836 13:17 Cartmel (WIN)
1.232219796 13:25 Chepstow (WIN)
1.232219926 13:37 Ripon (WIN)
1.232219887 13:42 Epsom (WIN)
1.232219971 13:47 Southwell (WIN)
1.232219843 13:52 Cartmel (WIN)
1.232219803 14:00 Chepstow (WIN)
1.232219931 14:12 Ripon (WIN)
1.232219894 14:17 Epsom (WIN)
1.232219978 14:22 Southwell (WIN)
1.232219856 14:27 Cartmel (WIN)
1.232219811 14:35 Chepstow (WIN)
1.232219938 14:47 Ripon (WIN)
1.232219901 14:52 Epsom (WIN)
1.232219984 14:57 Southwell (WIN)
1.232219850 15:02 Cartmel (WIN)
1.232219818 15:10 Chepstow (WIN)
1.232219945 15:20 Ripon (WIN)
1.232219907 15:25 Epsom (WIN)
1.232219991 15:30 Southwell (WIN)
1.232219862 15:40 Cartmel (WIN)
1.232219824 15:45 Chepstow (WIN)
1.232219951 15:55 Ripon (WIN)
1.232219914 16:00 Epsom (WIN)
1.232219998 16:05 Southwell (WIN)
1.232219869 16:15 Cartmel (WIN)
1.232219830 16:20 Chepstow (WIN)
1.232219958 1

In [7]:
import pandas as pd

# Initialize an empty list to store market and runner details
data = []

# Iterate over each market in the market catalogue
for market in results:
    market_id = market.market_id
    market_name = market.market_name
    market_start_time = market.market_start_time
    market_type = market.description.market_type

    # Extract the location/venue of the race
    event = market.event
    race_location = event.venue if event else "Unknown Location"
    
    # Iterate over each runner in the market
    for runner in market.runners:
        runner_id = runner.selection_id
        runner_name = runner.runner_name
        
        # Append a dictionary with all relevant data to the list
        data.append({
            'market_id': market_id,
            'market_start_time': market_start_time,
            'venue': race_location,
            'market_type': market_type,
            'selection_id': runner_id,
            'horse_name': runner_name,
        })

# Convert the list of dictionaries to a pandas DataFrame
df1 = pd.DataFrame(data)

# Display the DataFrame
print(df1)


       market_id   market_start_time      venue market_type  selection_id  \
0    1.232219790 2024-08-26 12:50:00   Chepstow         WIN      57783469   
1    1.232219790 2024-08-26 12:50:00   Chepstow         WIN      25769039   
2    1.232219790 2024-08-26 12:50:00   Chepstow         WIN      41063653   
3    1.232219790 2024-08-26 12:50:00   Chepstow         WIN      10028599   
4    1.232219790 2024-08-26 12:50:00   Chepstow         WIN        135704   
..           ...                 ...        ...         ...           ...   
336  1.232220010 2024-08-26 17:05:00  Southwell         WIN      41503353   
337  1.232220010 2024-08-26 17:05:00  Southwell         WIN      61804050   
338  1.232220010 2024-08-26 17:05:00  Southwell         WIN      69557275   
339  1.232220010 2024-08-26 17:05:00  Southwell         WIN      20591614   
340  1.232220010 2024-08-26 17:05:00  Southwell         WIN      20264937   

             horse_name  
0            Outer Edge  
1        Some Nightmare

# I need to get the a list of id's from the market_id column and then create a loop for the list_market_book 

In [8]:
market_id_list = df1['market_id'].drop_duplicates().to_list()
len(market_id_list)

35

In [9]:
# Define a delay between requests
delay = 0.05  # Delay in seconds

# Loop through market IDs and fetch data with a delay
market_books = []
for market_id in market_id_list:
    market_book = trading.betting.list_market_book(
        market_ids=[market_id],
        price_projection=betfairlightweight.filters.price_projection(
            price_data=betfairlightweight.filters.price_data(ex_all_offers=True)
        ),
    )
    market_books.append(market_book)
    
    # Print or process the market_book as needed
    print(market_book)
    
    # Sleep to avoid overloading the API
    time.sleep(delay)

[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]


In [10]:
# Example: if market_books is a list of lists of MarketBook objects
data = []

# Iterate over each list of MarketBook objects
for book_list in market_books:
    # Iterate over each MarketBook object in the current list
    for market_book in book_list:
        market_id = market_book.market_id
        status = market_book.status
        total_matched = market_book.total_matched

        # Iterate over each runner in the market
        for runner in market_book.runners:
            runner_id = runner.selection_id
            runner_status = runner.status
            last_price_traded = runner.last_price_traded
            runner_total_matched = runner.total_matched
            
            # Append a dictionary with all relevant data to the list
            data.append({
                'market_id': market_id,
                'status': status,
                'total_matched': total_matched,
                'selection_id': runner_id,
                'status': runner_status,
                'morning_price': last_price_traded,
            })



# Convert the list of dictionaries into a DataFrame
df2 = pd.DataFrame(data)

df2 = df2[['market_id', 'selection_id', 'status', 'morning_price', 'total_matched']]
df2

,market_id,selection_id,status,morning_price,total_matched
0,1.232219790,57783469,ACTIVE,5.7,7509.96
1,1.232219790,25769039,ACTIVE,5.9,7509.96
2,1.232219790,41063653,ACTIVE,5.7,7509.96
3,1.232219790,10028599,ACTIVE,6.8,7509.96
4,1.232219790,135704,ACTIVE,8.6,7509.96
...,...,...,...,...,...
336,1.232220010,41503353,ACTIVE,7.6,5496.42
337,1.232220010,61804050,ACTIVE,9.8,5496.42
338,1.232220010,69557275,ACTIVE,11.0,5496.42
339,1.232220010,20591614,ACTIVE,18.0,5496.42


# join df1 and df2

In [11]:
df3 = df1.merge(df2, on = ['market_id', 'selection_id'], how = 'left')

df3['market_id'] = df3['market_id'].astype(str)


current_date = datetime.now().strftime('%Y-%m-%d')
file_name = f"{current_date}_data.csv"


df3.to_csv('../data/daily/' + file_name, index=False)
df3

,market_id,market_start_time,venue,market_type,selection_id,horse_name,status,morning_price,total_matched
0,1.232219790,2024-08-26 12:50:00,Chepstow,WIN,57783469,Outer Edge,ACTIVE,5.7,7509.96
1,1.232219790,2024-08-26 12:50:00,Chepstow,WIN,25769039,Some Nightmare,ACTIVE,5.9,7509.96
2,1.232219790,2024-08-26 12:50:00,Chepstow,WIN,41063653,Notre Maison,ACTIVE,5.7,7509.96
3,1.232219790,2024-08-26 12:50:00,Chepstow,WIN,10028599,So Smart,ACTIVE,6.8,7509.96
4,1.232219790,2024-08-26 12:50:00,Chepstow,WIN,135704,Revenue,ACTIVE,8.6,7509.96
...,...,...,...,...,...,...,...,...,...
336,1.232220010,2024-08-26 17:05:00,Southwell,WIN,41503353,Kissininthebackrow,ACTIVE,7.6,5496.42
337,1.232220010,2024-08-26 17:05:00,Southwell,WIN,61804050,Freds Mate,ACTIVE,9.8,5496.42
338,1.232220010,2024-08-26 17:05:00,Southwell,WIN,69557275,Sydney Seabreeze,ACTIVE,11.0,5496.42
339,1.232220010,2024-08-26 17:05:00,Southwell,WIN,20591614,Defence Treaty,ACTIVE,18.0,5496.42
